# Advanced Interface features

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
# Install required libraries for Transformers, datasets, evaluation, and Gradio interface
!uv pip install datasets evaluate transformers[sentencepiece]
!uv pip install gradio

In [ ]:
# Stateful chatbot interface - demonstrates conversation memory with state management
# This creates a chatbot that remembers the conversation history
import random
import gradio as gr


def chat(message, history):
    # Initialize history if None (first message)
    history = history or []
    
    # Simple rule-based chatbot responses based on message patterns
    if message.startswith("How many"):
        response = random.randint(1, 10)  # Random number for "how many" questions
    elif message.startswith("How"):
        response = random.choice(["Great", "Good", "Okay", "Bad"])  # Random quality rating
    elif message.startswith("Where"):
        response = random.choice(["Here", "There", "Somewhere"])  # Random location
    else:
        response = "I don't know"  # Default response
    
    # Add the new exchange to conversation history
    # Each entry is a tuple: (user_message, bot_response)
    history.append((message, response))
    
    # Return updated history twice: once for display, once for state persistence
    return history, history


# Create chatbot interface with state management
iface = gr.Interface(
    chat,
    ["text", "state"],     # Inputs: current message + conversation state
    ["chatbot", "state"],  # Outputs: chat display + updated state
    allow_screenshot=False,  # Disable screenshot functionality
    allow_flagging="never",  # Disable user flagging of responses
)
iface.launch()

In [ ]:
# Image classification with interpretation features
# This demonstrates Gradio's built-in model interpretation capabilities
import requests
import tensorflow as tf
import gradio as gr

# Load pre-trained MobileNetV2 model for image classification
# MobileNetV2 is optimized for mobile/edge devices while maintaining good accuracy
inception_net = tf.keras.applications.MobileNetV2()

# Download human-readable labels for ImageNet classes
# ImageNet has 1000 object categories, each with a descriptive label
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")


def classify_image(inp):
    # Preprocess input image for MobileNetV2
    # Reshape to batch format: (batch_size, height, width, channels)
    inp = inp.reshape((-1, 224, 224, 3))
    
    # Apply MobileNetV2-specific preprocessing (normalization)
    inp = tf.keras.applications.mobilenet_v2.preprocess_input(inp)
    
    # Get model predictions for all 1000 ImageNet classes
    prediction = inception_net.predict(inp).flatten()
    
    # Return dictionary mapping class labels to confidence scores
    # This format is required for Gradio's Label output component
    return {labels[i]: float(prediction[i]) for i in range(1000)}


# Configure input and output components
image = gr.Image(shape=(224, 224))  # Fixed size image input matching model requirements
label = gr.Label(num_top_classes=3)  # Show top 3 predictions with confidence scores

title = "Gradio Image Classification + Interpretation Example"

# Create interface with built-in interpretation
gr.Interface(
    fn=classify_image, 
    inputs=image, 
    outputs=label, 
    interpretation="default",  # Enable automatic model interpretation
    title=title
).launch()